In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
model.invoke("최근 로제가 발표한 신곡은 무엇인가요?")

AIMessage(content='2023년 10월 기준으로 로제가 발표한 신곡은 "GONE"입니다. 이 곡은 그녀의 솔로 활동의 일환으로 출시되었으며, 많은 팬들에게 호평을 받았습니다. 보다 정확한 정보나 새로운 발표가 있는지 확인하고 싶다면, 최신 소식을 찾아보는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 19, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpVOVs7I1wjeAcTBAULfmfl7aGIyn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f47b4710-5750-492a-841a-33817272bca2-0', usage_metadata={'input_tokens': 19, 'output_tokens': 73, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
# 덕덕고 설치
%pip install -U duckduckgo-search

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 덕덕고 검색 테스트
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(results_separator=';\n')
docs = search.invoke("최근 로제가 발표한 신곡은 무엇인가요?")

print(docs)

snippet: 20일 (현지시간) 공개된 최신 차트에 따르면 로제가 팝스타 브루노 마스와 호흡을 맞춘 히트곡 '아파트' (APT.)는 영국 오피셜 싱글 차트 '톱 100'에서 전주보다 한 계단 하락한 5위로 9주 차트에 머물렀다. '아파트'는 이 차트에 4위로 처음 진입한 이후 2위, 5주 연속 3위, 4위, 5위를 기록하는 등 한 번도 '톱 파이브' 밖으로 밀려나지 않고 인기를 누리는 중이다. 이 곡이 수록된 로제의 첫 솔로 정규앨범 '로지'는 오피셜 앨범 차트 '톱 100'에서 전주보다 45위 하락한 49위로 2주 연속 이름을 올렸다. [JYP엔터테인먼트 제공. 재판매 및 DB 금지], title: 블랙핑크 로제, 솔로 1집으로 영국 싱글 5위·앨범 49위 - 연합뉴스, link: https://www.yna.co.kr/view/AKR20241221022400005;
snippet: 더블랙레이블은 22일 공식 SNS를 통해 로제의 새 싱글 'number one girl' 음원과 뮤직비디오 공개 소식을 전했다. 이는 'APT.'와 마찬가지로 오는 12월 6일 발매되는 로제의 첫 번째 정규 앨범 'rosie'에도 수록될 예정이다. 'rosie'는 총 12곡으로 구성되어 있으며, 로제는 이번 앨범에서 전곡 작사·작곡에 참여해 여러 장르를 아우르는 자신만의 음악 세계를 보여줄 예정이다. 특히 이번 신곡 'number one girl'은 로제가 팬들을 생각하며 써내려간 곡으로, 자신의 실제 경험을 녹여내기도 한 만큼 보다 진솔한 로제만의 이야기를 담고 있다., title: [오늘가요] "내가 특별하다고 말해줘"…로제, '넘버 원 걸' 발표, link: https://www.joynews24.com/view/1786189;
snippet: 걸그룹 블랙핑크의 멤버 로제가 22일 새 싱글 '넘버 원 걸 (Number one girl)'을 발표했다. 넘버 원 걸은 로제가 내달 6일에 발매하는 첫 정규앨범 '로지 (rosie)'의 선공개 곡이다. 소속사 더블랙레이블에 따르

In [4]:
# 확장 쿼리 구현
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "아래 context에 기반하여 사용자의 질문에 답변하라.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# LCEL 방식으로 바로 gpt와 연결
document_chain = question_answering_prompt | model

In [5]:
# 메시지 저장하고 답변 생성 후 출력하기
from langchain.memory import ChatMessageHistory

# 채팅 메시지를 저장할 메모리 객체 생성
chat_history = ChatMessageHistory()
# 사용자의 질문을 메모리에 저장
chat_history.add_user_message("최근 로제가 발표한 신곡은 무엇인가요?")

# 문서 검색하고 답변 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변을 메모리에 저장
chat_history.add_ai_message(answer)

print(answer)

content="로제가 최근 발표한 신곡은 '넘버 원 걸 (Number one girl)'입니다. 이 곡은 로제의 첫 번째 정규 앨범 '로지 (rosie)'의 선공개 곡으로, 브루노 마스가 프로듀서로 참여하였습니다." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 974, 'total_tokens': 1038, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BpVOZrrc88BMW9zcCpoTuf2mZELdI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--06319780-4764-49df-835f-7fd598d19ba2-0' usage_metadata={'input_tokens': 974, 'output_tokens': 64, 'total_tokens': 1038, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 검색 기능에 옵션 설정

In [23]:
# 최근 일주일 뉴스만 검색하기
# DuckDuckGo API 래퍼를 사용하여 검색할 때 검색 매개변수를 설정하는 클래스 import
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# 한국 지역("kr-kr")을 기준, 최근 일주일("w") 내의 검색 결과를 가져오도록 초기화
wrapper = DuckDuckGoSearchAPIWrapper(region="kr-kr", time="w")

In [24]:
# 뉴스 소스 위주로 검색하도록 지정하기
# 검색 기능을 위한 DuckDuckGoSearchResults 초기화
search = DuckDuckGoSearchResults(
    api_wrapper=wrapper, # 앞에서 정의한 wrapper 사용
    source="news", 
    results_separator=";\n", # 결과 항목 사이에 구분자 사용 - 세미콜론과 줄 바꿈
)

In [27]:
# 검색 결과 반환, 출력
# "로제의 신곡 APT에 대한 반응"을 검색하고 결과를 docs에 저장
docs = search.invoke("로제의 신곡 APT에 대한 반응")

# 검색 결과 출력
print(docs)

DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit

In [ ]:
# 특정 웹 사이트에서 검색하기 이거 안하는 게 좋겠다.
# ytn.co.kr 에서 검색
docs = search.invoke("site:k-trendynews.com 로제의 노래 APT")
print(docs)

### 기사 링크 가져오기

In [21]:
# 검색 결과의 링크들을 저장할 빈 리스트 초기화
links = []

# 검색 결과를 세미콜론과 줄 바꿈 기준으로 분리하고, 각 결과 항목에서 링크 추출
for doc in docs.split(";\n"):
    print(doc)
    if "link:" in doc:
        link = doc.split("link:")[1].strip()
        links.append(link)
    elif "link:" not in doc:
        print("검색 결과가 없습니다.")

# 모든 링크를 출력
print(links)


검색 결과가 없습니다.
[]
